# Import Labraries

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Connection To Database

In [54]:
conn = sqlite3.connect("Upi_Digital_Payment.db")
cursor = conn.cursor()


# Load Dataset


In [55]:
df = pd.read_csv("upi_digital_payment_advanced.csv")
df.to_sql("Digital_Payment",conn,if_exists="append",index=False)

5000

# Dataset Features

In [56]:
print("Size of DataFrame:\n", df.size)
print("Shape of DataFrame:\n", df.shape)

print("\nInfo of DataFrame:")
df.info()

print("\nDescription of DataFrame:\n")
print(df.describe())

print("\nColumns:\n", df.columns)

Size of DataFrame:
 45000
Shape of DataFrame:
 (5000, 9)

Info of DataFrame:
<class 'pandas.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    5000 non-null   str    
 1   user_id           5000 non-null   str    
 2   city              5000 non-null   str    
 3   transaction_date  5000 non-null   str    
 4   amount            5000 non-null   float64
 5   payment_app       5000 non-null   str    
 6   transaction_type  5000 non-null   str    
 7   device_type       5000 non-null   str    
 8   is_fraud          5000 non-null   int64  
dtypes: float64(1), int64(1), str(7)
memory usage: 351.7 KB

Description of DataFrame:

             amount     is_fraud
count   5000.000000  5000.000000
mean    1995.619530     0.033000
std     2029.263919     0.178654
min        0.670000     0.000000
25%      557.285000     0.000000
50%     1363.480000     0

# Dataset Cleaning

In [57]:
print("Null Values From Dataset:\n",df.isna().sum())
print("Duplicate Values From Dateset:",df.duplicated().sum())


Null Values From Dataset:
 transaction_id      0
user_id             0
city                0
transaction_date    0
amount              0
payment_app         0
transaction_type    0
device_type         0
is_fraud            0
dtype: int64
Duplicate Values From Dateset: 0


In [58]:
# Convert Date Column into Datetime

df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df["Month"] = df["transaction_date"].dt.month_name()
df["Time"] = df["transaction_date"].dt.strftime("%I %p")
print(df[["transaction_date","Month","Time"]])

        transaction_date     Month   Time
0    2024-12-14 21:20:20  December  09 PM
1    2024-03-28 12:15:31     March  12 PM
2    2024-12-09 17:29:15  December  05 PM
3    2024-10-03 23:05:04   October  11 PM
4    2024-02-28 23:27:58  February  11 PM
...                  ...       ...    ...
4995 2024-08-28 04:21:40    August  04 AM
4996 2024-06-08 16:34:18      June  04 PM
4997 2024-01-20 15:09:58   January  03 PM
4998 2024-03-04 00:15:44     March  12 AM
4999 2024-06-15 19:22:00      June  07 PM

[5000 rows x 3 columns]


# Check Dataload In Created Table

In [59]:
query = "PRAGMA table_info(Digital_Payment)"
info = pd.read_sql(query,conn)
print(info)

    cid              name       type  notnull dflt_value  pk
0     0    transaction_id       TEXT        0       None   0
1     1           user_id       TEXT        0       None   0
2     2              city       TEXT        0       None   0
3     3  transaction_date  TIMESTAMP        0       None   0
4     4            amount       REAL        0       None   0
5     5       payment_app       TEXT        0       None   0
6     6  transaction_type       TEXT        0       None   0
7     7       device_type       TEXT        0       None   0
8     8          is_fraud    INTEGER        0       None   0
9     9             Month       TEXT        0       None   0
10   10              Time       TIME        0       None   0


# Average Fraud Analysis

In [60]:
query = """SELECT transaction_type,user_id,city,
payment_app,device_type,AVG(is_fraud)
FROM Digital_Payment
GROUP BY transaction_type,user_id,city,
payment_app,device_type
ORDER BY AVG(is_fraud) DESC
"""

top_transactiontype_fraud = pd.read_sql(query,conn)
print(top_transactiontype_fraud)


     transaction_type   user_id       city payment_app device_type  \
0        Bill Payment  USER1258    Lucknow       Paytm     Android   
1        Bill Payment  USER1480    Kolkata  Amazon Pay     Android   
2        Bill Payment  USER1492    Kolkata     PhonePe         iOS   
3        Bill Payment  USER1561       Pune       Paytm         iOS   
4        Bill Payment  USER1711  Ahmedabad     PhonePe         Web   
...               ...       ...        ...         ...         ...   
4994         Recharge  USER9970  Bangalore       Paytm         iOS   
4995         Recharge  USER9973  Bangalore  Amazon Pay     Android   
4996         Recharge  USER9976  Bangalore       Paytm         Web   
4997         Recharge  USER9988     Mumbai       Paytm     Android   
4998         Recharge  USER9992  Bangalore  Amazon Pay         iOS   

      AVG(is_fraud)  
0               1.0  
1               1.0  
2               1.0  
3               1.0  
4               1.0  
...             ...  
4994 

#  Fraud Analysis according to time

In [61]:
df.to_sql("Digital_Payment",conn,if_exists="replace",index=False)

5000

In [62]:
query = """SELECT Time,AVG(is_fraud)
 FROM Digital_Payment 
 GROUP BY Time
 ORDER BY AVG(is_fraud) DESC
 LIMIT 5
"""
time_fraud = pd.read_sql(query,conn)
print(time_fraud)

    Time  AVG(is_fraud)
0  12 AM       0.096639
1  02 AM       0.090476
2  11 PM       0.085859
3  04 AM       0.080357
4  03 AM       0.052133


# Payment App wise Fraud Analysis

In [66]:
query = """SELECT payment_app,Avg(is_fraud) AS fraud_rate
FROM digital_Payment
GROUP BY payment_app
ORDER BY AVG(is_fraud) DESC
LIMIT 5
"""
Fraud_trending_payment_app = pd.read_sql(query,conn)
print(Fraud_trending_payment_app)

  payment_app  fraud_rate
0  Amazon Pay    0.038736
1       Paytm    0.034518
2        BHIM    0.033898
3     PhonePe    0.032787
4  Google Pay    0.025151


# City-wise Fraud Analysis

In [65]:
query = """SELECT city ,AVG(is_fraud) as fraud_rate
FROM Digital_Payment
GROUP BY city
Order by fraud_rate DESC
LIMIT 5
"""
Fraud_Trending_City = pd.read_sql(query,conn)
print(Fraud_Trending_City)

        city  fraud_rate
0    Chennai    0.054000
1  Ahmedabad    0.040685
2  Bangalore    0.035644
3  Hyderabad    0.033663
4      Delhi    0.033663
